# Download Packages

This notebook downloads Linux kernel packages using links gathered in the previous notebook.

Note: Skip this notebook if you already have the dataset. 


In [1]:
%reload_ext autoreload
%autoreload 2

The following cell downloads approximately 23GB of data. The download time will vary based on your network connection speed and could take several minutes to complete.

In [2]:
from pathlib import Path

from depsurf import Version, manage_result_path, system


urls = [
    # Generic, amd64, first revision for 16.04 - 24.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-4.4.0-21-generic-dbgsym_4.4.0-21.37_amd64.ddeb",  # 16.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-4.8.0-22-generic-dbgsym_4.8.0-22.24_amd64.ddeb",  # 16.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-4.10.0-19-generic-dbgsym_4.10.0-19.21_amd64.ddeb",  # 17.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-4.13.0-16-generic-dbgsym_4.13.0-16.19_amd64.ddeb",  # 17.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-4.15.0-20-generic-dbgsym_4.15.0-20.21_amd64.ddeb",  # 18.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-4.18.0-10-generic-dbgsym_4.18.0-10.11_amd64.ddeb",  # 18.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.0.0-13-generic-dbgsym_5.0.0-13.14_amd64.ddeb",  # 19.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.3.0-18-generic-dbgsym_5.3.0-18.19_amd64.ddeb",  # 19.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_amd64.ddeb",  # 20.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.8.0-25-generic-dbgsym_5.8.0-25.26_amd64.ddeb",  # 20.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.11.0-16-generic-dbgsym_5.11.0-16.17_amd64.ddeb",  # 21.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.13.0-19-generic-dbgsym_5.13.0-19.19_amd64.ddeb",  # 21.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.15.0-25-generic-dbgsym_5.15.0-25.25_amd64.ddeb",  # 22.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.19.0-21-generic-dbgsym_5.19.0-21.21_amd64.ddeb",  # 22.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.2.0-20-generic-dbgsym_6.2.0-20.20_amd64.ddeb",  # 23.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.5.0-9-generic-dbgsym_6.5.0-9.9_amd64.ddeb",  # 23.10
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-31-generic-dbgsym_6.8.0-31.31_amd64.ddeb",  # 24.04
    # Revisions for 20.04
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-51-generic-dbgsym_5.4.0-51.56_amd64.ddeb",  # 51
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-75-generic-dbgsym_5.4.0-75.84_amd64.ddeb",  # 75
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-100-generic-dbgsym_5.4.0-100.113_amd64.ddeb",  # 100
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-125-generic-dbgsym_5.4.0-125.141_amd64.ddeb",  # 125
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-175-generic-dbgsym_5.4.0-175.195_amd64.ddeb",  # 175
    # Architectures for 20.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_arm64.ddeb",  # arm64
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-5.4.0-26-generic-dbgsym_5.4.0-26.30_armhf.ddeb",  # armhf
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-5.4.0-24-generic-dbgsym_5.4.0-24.28_riscv64.ddeb",  # riscv64
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_ppc64el.ddeb",  # ppc64el
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_s390x.ddeb",  # s390x
    # Flavors for 20.04
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-lowlatency-dbgsym_5.4.0-26.30_amd64.ddeb",  # lowlatency
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-1010-azure-dbgsym_5.4.0-1010.10_amd64.ddeb",  # azure
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-5.4.0-1009-aws-dbgsym_5.4.0-1009.9_amd64.ddeb",  # aws
    "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-1009-gcp-dbgsym_5.4.0-1009.9_amd64.ddeb",  # gcp
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-1009-oracle-dbgsym_5.4.0-1009.9_amd64.ddeb",  # oracle
    # Architectures for 24.04
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-31-generic-dbgsym_6.8.0-31.31_arm64.ddeb",  # arm64
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-6.8.0-31-generic-dbgsym_6.8.0-31.31_armhf.ddeb",  # armhf
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-6.8.0-31-generic-dbgsym_6.8.0-31.31_ppc64el.ddeb",  # ppc64el
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-31-generic-dbgsym_6.8.0-31.31_s390x.ddeb",  # s390x
    # Flavors for 24.04
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-31-lowlatency-dbgsym_6.8.0-31.31.1_amd64.ddeb",  # lowlatency
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-1007-azure-dbgsym_6.8.0-1007.7_amd64.ddeb",  # azure
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-1008-aws-dbgsym_6.8.0-1008.8_amd64.ddeb",  # aws
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-1007-gcp-dbgsym_6.8.0-1007.7_amd64.ddeb",  # gcp
    # "https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-1005-oracle-dbgsym_6.8.0-1005.5_amd64.ddeb",  # oracle
]


@manage_result_path
def download(url: str, result_path: Path):
    system(f"wget -O {result_path} {url}")


for dbgsym_url in urls:
    v = Version.from_path(dbgsym_url)

    # download the dbgsym package
    download(dbgsym_url, result_path=v.dbgsym_download_path)

    # download the image package
    vmlinuz_url = dbgsym_url.replace("-dbgsym", "").replace(".ddeb", ".deb")
    download(vmlinuz_url, result_path=v.image_download_path)

    url = vmlinuz_url.replace("unsigned-", "")
    # download the modules package
    if v.version_tuple >= (4, 15, 0):
        modules_url = url.replace("image-", "modules-")
        download(modules_url, result_path=v.modules_download_path)

    # download the buildinfo package
    if v.version_tuple >= (5, 0, 0):
        buildinfo_url = url.replace("image-", "buildinfo-")
        download(buildinfo_url, result_path=v.buildinfo_download_path)

[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/dbgsym/4.4.0-21-generic-amd64.deb
[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/image/4.4.0-21-generic-amd64.deb
[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/dbgsym/4.8.0-22-generic-amd64.deb
[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/image/4.8.0-22-generic-amd64.deb
[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/dbgsym/4.10.0-19-generic-amd64.deb
[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/image/4.10.0-19-generic-amd64.deb
[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/dbgsym/4.13.0-16-generic-amd64.deb
[    decorator.py:12 ] INFO: download           Skipped /users/szhong/DepSurf/data/download/image/4.13.0-16-generic-am

In [4]:
!du -h data/download

22G	data/download/dbgsym
453M	data/download/modules
6.7M	data/download/buildinfo
279M	data/download/image
23G	data/download
